In [ ]:
import urllib.request
import requests
import pandas as pd


In [ ]:
confirmation_url = "https://www.microsoft.com/en-us/download/confirmation.aspx?id=56519"
import json
import requests
from bs4 import BeautifulSoup
 
 
# Making a GET request
r = requests.get(confirmation_url)
 
# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')

url=""
#url = "https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20220404.json"
# find all the anchor tags with "href"
for link in soup.find_all('a'):
    if "ServiceTags_Public" in link.get('href'):
        url=link.get('href')


data = json.loads(requests.get(url).text)

df_tags = pd.json_normalize(data, record_path=['values'], meta=['changeNumber','cloud'],errors='ignore')

In [ ]:
df = pd.DataFrame()

PACKAGE="WS5"
API_KEY="D54MEY6ZMD"

ipranges=list(df_tags[df_tags['properties.systemService']=="PowerBI"]['properties.addressPrefixes'])[0]

response=""
# don't need to write to a local file anymore
#with open('iprange_doc.json', 'w') as f:
for ip in ipranges:
    my_request = f"https://api.ip2location.com/v2/?ip={ip.split('/')[0]}&key={API_KEY}&package={PACKAGE}"
    r = urllib.request.urlopen(my_request)
    response = r.read().decode('utf-8')
    df_temp = pd.json_normalize(json.loads(response))
    df_temp['CIDR']=ip
    
    df=pd.concat([df,df_temp])

In [ ]:
df.tail(20)

In [ ]:
df.to_excel("/mnt/c/Users/brcampb/Documents/powerbi_ipranges.xlsx")
df.to_excel("powerbi_ipranges.xlsx")

In [1]:
import pandas as pd
df = pd.read_excel("powerbi_ipranges.xlsx")

In [40]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.identity import ClientSecretCredential
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.storage import StorageManagementClient
from datetime import datetime, timedelta
import time

from azure.mgmt.storage.models import (
    StorageAccountCreateParameters,
    NetworkRuleSet,
    IPRule,
    StorageAccountUpdateParameters,
    Sku,
    SkuName,
    Kind
)

In [34]:
import os
subscription_id = os.getenv("SUBSCRIPTION_ID")

credentials = ServicePrincipalCredentials(
    client_id = 'a888b9fe-38ff-4551-844f-7416e1cbb89f',
    secret=os.getenv("ECOLAB_ADF_SP_SECRET"),
    tenant=os.getenv("TENANT_ID")
)

blob_credentials = ClientSecretCredential(
    client_id = 'a888b9fe-38ff-4551-844f-7416e1cbb89f',
    client_secret=os.getenv("ECOLAB_ADF_SP_SECRET"),
    tenant_id=os.getenv("TENANT_ID")
)

GROUP_NAME = 'ecolab-rg'
STORAGE_ACCOUNT_NAME = 'ecolabgen2'

#credentials = DefaultAzureCredential(exclude_interactive_browser_credential=False)

resource_client = ResourceManagementClient(credentials, subscription_id)
storage_client = StorageManagementClient(credentials, subscription_id)

In [35]:
storage_account = storage_client.storage_accounts.get_properties("ecolab-rg","ecolabgen2")

In [109]:
ip = df[df['region_name']=='Iowa']['CIDR']

str_ip_rules = ""

ip_ranges2 = []

# /31 and /32 are not allowed to be entered as IP Ranges using CIDR
# must be entered as an IP
for address in ip:
    if ":" not in address:
        if "31" in address:
            val = address.replace("/31","")
            str_ip_rules += f"IPRule(ip_address_or_range='{val}', action='Allow')|"
            ip_ranges2.append(IPRule(ip_address_or_range=val, action='Allow'))
            val = val.rsplit(".",1)
            val = val[0] + "." + str(int(val[1])+1)
            str_ip_rules += f"IPRule(ip_address_or_range='{val}', action='Allow')|"
            ip_ranges2.append(IPRule(ip_address_or_range=val, action='Allow'))


        else:
            str_ip_rules += f"IPRule(ip_address_or_range='{address}', action='Allow')|"
            ip_ranges2.append(IPRule(ip_address_or_range=address, action='Allow'))


In [111]:
str_ip_rules=str_ip_rules[0:-1]
ip_rules = list(str_ip_rules.split("|"))

### Working Example

This shows what it looks like if just entering by hand

In [92]:
ips=['20.37.156.200/30','20.37.156.240/28']

storage_account = storage_client.storage_accounts.update(
    GROUP_NAME, STORAGE_ACCOUNT_NAME,
    StorageAccountUpdateParameters (
        network_rule_set = NetworkRuleSet(
            default_action='Deny',
            ip_rules = [
                  IPRule(ip_address_or_range='20.37.156.200/30', action='Allow')
                , IPRule(ip_address_or_range='20.37.156.240/28', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.124', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.125', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.208/30', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.216/29', action='Allow')
                , IPRule(ip_address_or_range='20.45.192.224/28', action='Allow')
                , IPRule(ip_address_or_range='20.45.242.48/29', action='Allow')
                , IPRule(ip_address_or_range='20.46.15.56/30', action='Allow')
                , IPRule(ip_address_or_range='20.98.145.48/28', action='Allow')
                , IPRule(ip_address_or_range='20.98.145.64/30', action='Allow')
                , IPRule(ip_address_or_range='20.98.145.112/29', action='Allow')
                , IPRule(ip_address_or_range='20.98.146.0/27', action='Allow')
            ]
        )
    )
)

### This works

This is how to add the list of IPRules through dynamic coding

In [113]:
storage_account = storage_client.storage_accounts.update(
    GROUP_NAME, STORAGE_ACCOUNT_NAME,
    StorageAccountUpdateParameters (
        network_rule_set = NetworkRuleSet(
            default_action='Deny',
            ip_rules = ip_ranges2
        )
    )
)

StorageAccountUpdateParameters(*,  
**sku:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.Sku] = None,  
**tags:** Optional[Dict[str, str]] = None, identity: Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.Identity] = None,   
**kind:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.Kind]] = None, 
**custom_domain:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.CustomDomain] = None,  
**encryption:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.Encryption] = None,  
**access_tier:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.AccessTier]] = None,  **azure_files_identity_based_authentication:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.AzureFilesIdentityBasedAuthentication] = None,   
**enable_https_traffic_only:** Optional[bool] = None,  
**network_rule_set:** Optional[azure.mgmt.storage.v2019_04_01.models._models_py3.NetworkRuleSet] = None,  
**large_file_shares_state:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.LargeFileSharesState]] =None,  **allow_blob_public_access:** Optional[bool] = None,  
**minimum_tls_version:** Optional[Union[str, azure.mgmt.storage.v2019_04_01.models._storage_management_client_enums.MinimumTlsVersion]] = None,  **allow_shared_key_access:** Optional[bool] = None, **kwargs)

NetworkRuleSet(*,  
*default_action:* Union[str, azure.mgmt.storage.v2021_01_01.models._storage_management_client_enums.DefaultAction] = 'Allow',  
*bypass:* Optional[Union[str, azure.mgmt.storage.v2021_01_01.models._storage_management_client_enums.Bypass]] = 'AzureServices',  
*resource_access_rules:* Optional[List[azure.mgmt.storage.v2021_01_01.models._models_py3.ResourceAccessRule]] = None,  
*virtual_network_rules:* Optional[List[azure.mgmt.storage.v2021_01_01.models._models_py3.VirtualNetworkRule]] = None,   
*ip_rules:* Optional[List[azure.mgmt.storage.v2021_01_01.models._models_py3.IPRule]] = None, **kwargs)

In [ ]:
from azure.storage.blob import BlobServiceClient

service = BlobServiceClient(account_url="https://ecolabgen2.blob.core.windows.net/", credential=blob_credentials)
#service.create_container("testing1")